In [1]:
import rasterio
import pandas as pd
from osgeo import gdal
from pyproj import Proj, transform
import numpy as np
import math
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from scipy.stats import kde
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from matplotlib.ticker import MaxNLocator
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

from sklearn.inspection import permutation_importance
from decimal import Decimal
from IPython.display import display, HTML

import time

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


## Read files

In [2]:
# find index where contains non-zero
# group with same block and get average

In [ ]:
# June data

file_0611 = np.array([rasterio.open('0611_18_5000.tif').read(), rasterio.open('0611_18_10000.tif').read(), rasterio.open('0611_19_0.tif').read()])
file_0618 = np.array([rasterio.open('0618_19_5000.tif').read(), rasterio.open('0618_19_10000.tif').read(), rasterio.open('0618_20_5000.tif').read(), rasterio.open('0618_21_5000.tif').read()])

In [ ]:
# August data

file_0806 = np.array([rasterio.open('0806_15_0.tif').read(), rasterio.open('0806_15_5000.tif').read(), rasterio.open('0806_16_5000.tif').read(), rasterio.open('0806_17_0.tif').read()])
file_0819 = np.array([rasterio.open('0819_16_0.tif').read(), rasterio.open('0819_17_0.tif').read(), rasterio.open('0819_17_5000.tif').read(), rasterio.open('0819_18_0.tif').read(), rasterio.open('0819_18_5000.tif').read()])
file_0826 = np.array([rasterio.open('0826_17_5000.tif').read(), rasterio.open('0826_17_10000.tif').read(), rasterio.open('0826_18_0.tif').read(), rasterio.open('0826_19_0.tif').read(), rasterio.open('0826_19_5000.tif').read()])

In [3]:
# September data

file_0905 = np.array([rasterio.open('0905_14_0.tif').read(), rasterio.open('0905_14_5000.tif').read(), rasterio.open('0905_15_0.tif').read(), rasterio.open('0905_16_0.tif').read(), rasterio.open('0905_17_0.tif').read(), rasterio.open('0905_17_5000.tif').read()])
file_0919 = np.array([rasterio.open('0919_15_0.tif').read(), rasterio.open('0919_16_0.tif').read(), rasterio.open('0919_17_0.tif').read(), rasterio.open('0919_18_0.tif').read()])

In [4]:
# Label data

map_ = rasterio.open('yield_pdf_report.tif').read()
map_ = map_[0]

## Merge data into matrix

In [ ]:
# June data
# for each band each day, group overlap pixel in one group

total_0611 = []
total_0618 = []

for band in range(360):

    data_0611 = np.dstack((file_0611[0][band].reshape(254, 447, 1), file_0611[1][band].reshape(254, 447, 1)))
    data_0611 = np.dstack((data_0611, file_0611[2][band].reshape(254, 447, 1)))

    data_0618 = np.dstack((file_0618[0][band].reshape(254, 447, 1), file_0618[1][band].reshape(254, 447, 1)))
    data_0618 = np.dstack((data_0618, file_0618[1][band].reshape(254, 447, 1)))
    data_0618 = np.dstack((data_0618, file_0618[2][band].reshape(254, 447, 1)))
    data_0618 = np.dstack((data_0618, file_0618[3][band].reshape(254, 447, 1)))
    
    total_0611.append(data_0611)
    total_0618.append(data_0618)
    
total_0611 = np.array(total_0611)
total_0618 = np.array(total_0618)

In [ ]:
# August data
# for each band each day, group overlap pixel in one group

total_0806 = []
total_0819 = []
total_0826 = []

for band in range(360):
    data_0806 = np.dstack((file_0806[0][band].reshape(254, 447, 1),file_0806[1][band].reshape(254, 447, 1)))
    data_0806 = np.dstack((data_0806,file_0806[2][band].reshape(254, 447, 1)))
    data_0806 = np.dstack((data_0806,file_0806[3][band].reshape(254, 447, 1)))
    
    data_0819 = np.dstack((file_0819[0][band].reshape(254, 447, 1),file_0819[1][band].reshape(254, 447, 1)))
    data_0819 = np.dstack((data_0819,file_0819[2][band].reshape(254, 447, 1)))
    data_0819 = np.dstack((data_0819,file_0819[3][band].reshape(254, 447, 1)))
    data_0819 = np.dstack((data_0819,file_0819[4][band].reshape(254, 447, 1)))
    
    data_0826 = np.dstack((file_0826[0][band].reshape(254, 447, 1),file_0826[1][band].reshape(254, 447, 1)))
    data_0826 = np.dstack((data_0826,file_0826[2][band].reshape(254, 447, 1)))
    data_0826 = np.dstack((data_0826,file_0826[3][band].reshape(254, 447, 1)))
    data_0826 = np.dstack((data_0826,file_0826[4][band].reshape(254, 447, 1)))

    
    total_0806.append(data_0806)
    total_0819.append(data_0819)
    total_0826.append(data_0826)

total_0806 = np.array(total_0806)
total_0819 = np.array(total_0819)
total_0826 = np.array(total_0826)

In [5]:
# September data
# for each band each day, group overlap pixel in one group

start_time = time.time()

total_0905 = []
total_0919 = []

for band in range(360):
    data_0905 = np.dstack((file_0905[0][band].reshape(254, 447, 1),file_0905[1][band].reshape(254, 447, 1)))
    data_0905 = np.dstack((data_0905,file_0905[2][band].reshape(254, 447, 1)))
    data_0905 = np.dstack((data_0905,file_0905[3][band].reshape(254, 447, 1)))
    data_0905 = np.dstack((data_0905,file_0905[4][band].reshape(254, 447, 1)))
    data_0905 = np.dstack((data_0905,file_0905[5][band].reshape(254, 447, 1)))

    
    data_0919 = np.dstack((file_0919[0][band].reshape(254, 447, 1),file_0919[1][band].reshape(254, 447, 1)))
    data_0919 = np.dstack((data_0919,file_0919[2][band].reshape(254, 447, 1)))
    data_0919 = np.dstack((data_0919,file_0919[3][band].reshape(254, 447, 1)))

    
    total_0905.append(data_0905)
    total_0919.append(data_0919)

total_0905 = np.array(total_0905)
total_0919 = np.array(total_0919)

print("--- %s seconds ---" % (time.time() - start_time))

--- 3.7904200553894043 seconds ---


## Data cleaning: remove points where corresponding label is invalid

In [98]:
# find invalid index based on map_ (since we only consider points with not zero for yield as valid)
# June data

index_ = np.where(map_ == 0)

# replace data with 0 on invalid point

for band in range(360):
    total_0611[band][index_] = 0
    total_0618[band][index_] = 0

In [8]:
# August data

index_ = np.where(map_ == 0)

for band in range(360):
    total_0806[band][index_] = 0
    total_0819[band][index_] = 0
    total_0826[band][index_] = 0

In [6]:
# Stptember data

index_ = np.where(map_ == 0)

for band in range(360):
    total_0905[band][index_] = 0
    total_0919[band][index_] = 0

## Group data and take average to reduce noise

In [99]:
# june data

group = 5
x = int(254/group)
y = int(447/group)

June_result = []

for band in range(360):
    if ((band%10 == 0) and band != 0):
#         print("finish " + str(band))
    cur_0611 = total_0611[band]
    cur_0618 = total_0618[band]
    
    count = 0
    for i in range(x):
        for j in range(y):
            cur_spot = []
            temp_0611 = cur_0611[i*group:(i+1)*group, j*group:(j+1)*group].astype(Decimal).flatten()
            temp_0618 = cur_0618[i*group:(i+1)*group, j*group:(j+1)*group].astype(Decimal).flatten()
            
            mean_0611 = 0
            mean_0618 = 0

            if (np.all((temp_0611 != 0))):
                masked = np.ma.masked_equal(temp_0611, 0)
                mean_0611 = masked.mean(axis = 0)

            if (np.all((temp_0618 != 0))):
                masked = np.ma.masked_equal(temp_0618, 0)
                mean_0618 = masked.mean(axis = 0)

            cur_spot.append(mean_0611)
            cur_spot.append(mean_0618)
            
            if (band == 0):
                June_result.append(cur_spot[:2])
            else:
                June_result[count] += cur_spot[:2]
            count += 1

finish 10
finish 20
finish 30
finish 40
finish 50
finish 60
finish 70
finish 80
finish 90
finish 100
finish 110
finish 120
finish 130
finish 140
finish 150
finish 160
finish 170
finish 180
finish 190
finish 200
finish 210
finish 220
finish 230
finish 240
finish 250
finish 260
finish 270
finish 280
finish 290
finish 300
finish 310
finish 320
finish 330
finish 340
finish 350


In [9]:
# August data

group = 5
x = int(254/group)
y = int(447/group)

August_result = []

for band in range(360):
    if ((band%10 == 0) and band != 0):
#         print("finish " + str(band))
    cur_0806 = total_0806[band]
    cur_0819 = total_0819[band]
    cur_0826 = total_0826[band]
    
    count = 0
    for i in range(x):
        for j in range(y):
            cur_spot = []
            temp_0806 = cur_0806[i*group:(i+1)*group, j*group:(j+1)*group].astype(Decimal).flatten()
            temp_0819 = cur_0819[i*group:(i+1)*group, j*group:(j+1)*group].astype(Decimal).flatten()
            temp_0826 = cur_0826[i*group:(i+1)*group, j*group:(j+1)*group].astype(Decimal).flatten()
            
            mean_0806 = 0
            mean_0819 = 0
            mean_0826 = 0
            
            if (np.all((temp_0806 != 0))):
                masked = np.ma.masked_equal(temp_0806, 0)
                mean_0806 = masked.mean(axis = 0)
                
            if (np.all((temp_0819 != 0))):
                masked = np.ma.masked_equal(temp_0819, 0)
                mean_0819 = masked.mean(axis = 0)
                
            if (np.all((temp_0826 != 0))):
                masked = np.ma.masked_equal(temp_0826, 0)
                mean_0826 = masked.mean(axis = 0)

            cur_spot.append(mean_0806)
            cur_spot.append(mean_0819)
            cur_spot.append(mean_0826)
            
            if (band == 0):
                August_result.append(cur_spot)
            else:
                August_result[count] += cur_spot
            count += 1

finish 10
finish 20
finish 30
finish 40
finish 50
finish 60
finish 70
finish 80
finish 90
finish 100
finish 110
finish 120
finish 130
finish 140
finish 150
finish 160
finish 170
finish 180
finish 190
finish 200
finish 210
finish 220
finish 230
finish 240
finish 250
finish 260
finish 270
finish 280
finish 290
finish 300
finish 310
finish 320
finish 330
finish 340
finish 350


In [10]:
# September data

group = 5
x = int(254/group)
y = int(447/group)

September_result = []

for band in range(360):
    if ((band%10 == 0) and band != 0):
#         print("finish " + str(band))
    cur_0905 = total_0905[band]
    cur_0919 = total_0919[band]
    
    count = 0
    for i in range(x):
        for j in range(y):
            cur_spot = []
            temp_0905 = cur_0905[i*group:(i+1)*group, j*group:(j+1)*group].astype(Decimal).flatten()
            temp_0919 = cur_0919[i*group:(i+1)*group, j*group:(j+1)*group].astype(Decimal).flatten()
            
            mean_0905 = 0
            mean_0919 = 0
            
            if (np.all((temp_0905 != 0))):
                masked = np.ma.masked_equal(temp_0905, 0)
                mean_0905 = masked.mean(axis = 0)
                
            if (np.all((temp_0919 != 0))):
                masked = np.ma.masked_equal(temp_0919, 0)
                mean_0919 = masked.mean(axis = 0)

            cur_spot.append(mean_0905)
            cur_spot.append(mean_0919)
            
            if (band == 0):
                September_result.append(cur_spot)
            else:
                September_result[count] += cur_spot
            count += 1

finish 10
finish 20
finish 30
finish 40
finish 50
finish 60
finish 70
finish 80
finish 90
finish 100
finish 110
finish 120
finish 130
finish 140
finish 150
finish 160
finish 170
finish 180
finish 190
finish 200
finish 210
finish 220
finish 230
finish 240
finish 250
finish 260
finish 270
finish 280
finish 290
finish 300
finish 310
finish 320
finish 330
finish 340
finish 350


## Generate test data

In [13]:
Y = []
for i in range(x):
    for j in range(y):
        temp = map_[i*group:(i+1)*group, j*group:(j+1)*group].astype(Decimal).flatten()
        mean = 0
        if (np.all((temp != 0))):
            masked = np.ma.masked_equal(temp, 0)
            mean = masked.mean(axis = 0)
        Y.append(mean)

In [14]:
# sanity check

Y = np.array(Y)
print(Y.shape)
print(np.array(August_result).shape)
print(np.array(September_result).shape)

(4450,)
(4450, 1080)
(4450, 720)


## Data cleaning: remove blocks with all invalid data

In [114]:
# June data

June_drop_zero = []

for i in range(Y.shape[0]):
    if Y[i] != 0:
        Y_drop_zero.append(Y[i])
        June_drop_zero.append(June_result[i])

In [15]:
# August data

August_drop_zero = []
Y_drop_zero = []

for i in range(Y.shape[0]):
    if Y[i] != 0:
        Y_drop_zero.append(Y[i])
        August_drop_zero.append(August_result[i])

In [16]:
# September data

September_drop_zero = []

for i in range(Y.shape[0]):
    if Y[i] != 0:
        September_drop_zero.append(September_result[i])

In [30]:
# August + September data

Total_drop_zero = []

for i in range(len(August_drop_zero)):
    temp = np.copy(August_drop_zero[i])
    for j in range(len(September_drop_zero[i])):
        temp = np.append(temp, September_drop_zero[i][j])
    Total_drop_zero.append(temp)

## Random Forest Classifier

In [45]:
c_Y = []

for i in range(len(Y_drop_zero)):
    val = round(Y_drop_zero[i])
    c_Y.append(val)

In [116]:
c_June_x_train, c_June_x_test, c_June_y_train, c_June_y_test = train_test_split(June_drop_zero, c_Y, test_size = 0.3)
c_June_model = RandomForestClassifier(max_depth = 15, random_state = 0)
c_June_model.fit(c_June_x_train, c_June_y_train)
c_June_y_pred = c_June_model.predict(c_June_x_test)

In [115]:
c_August_x_train, c_August_x_test, c_August_y_train, c_August_y_test = train_test_split(August_drop_zero, c_Y, test_size = 0.3)
c_August_model = RandomForestClassifier(max_depth = 15, random_state = 0)
c_August_model.fit(c_August_x_train, c_August_y_train)
c_August_y_pred = c_August_model.predict(c_August_x_test)

In [117]:
c_September_x_train, c_September_x_test, c_September_y_train, c_September_y_test = train_test_split(September_drop_zero, c_Y, test_size = 0.3)
c_September_model = RandomForestClassifier(max_depth = 15, random_state = 0)
c_September_model.fit(c_September_x_train, c_September_y_train)
c_September_y_pred = c_September_model.predict(c_September_x_test)

In [46]:
c_Total_x_train, c_Total_x_test, c_Total_y_train, c_Total_y_test = train_test_split(Total_drop_zero, c_Y, test_size = 0.3)
c_Total_model = RandomForestClassifier(max_depth = 15, random_state = 0)
c_Total_model.fit(c_Total_x_train, c_Total_y_train)
c_Total_y_pred = c_Total_model.predict(c_Total_x_test)

In [68]:
print(classification_report(c_June_y_test, c_June_y_pred))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         1
           2       0.00      0.00      0.00         4
           3       0.00      0.00      0.00        26
           4       0.00      0.00      0.00        74
           5       0.00      0.00      0.00       249
           6       0.53      1.00      0.69       584
           7       0.00      0.00      0.00       161

    accuracy                           0.53      1099
   macro avg       0.08      0.14      0.10      1099
weighted avg       0.28      0.53      0.37      1099



c:\users\windows10\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\windows10\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\windows10\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [67]:
print(classification_report(c_August_y_test, c_August_y_pred))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         3
           2       0.00      0.00      0.00         2
           3       0.00      0.00      0.00        28
           4       0.00      0.00      0.00        88
           5       0.57      0.11      0.19       256
           6       0.54      0.94      0.68       571
           7       0.26      0.06      0.10       151

    accuracy                           0.53      1099
   macro avg       0.19      0.16      0.14      1099
weighted avg       0.45      0.53      0.41      1099



c:\users\windows10\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\windows10\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\windows10\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [118]:
print(classification_report(c_September_y_test, c_September_y_pred))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         5
           3       0.00      0.00      0.00        35
           4       0.33      0.02      0.04        91
           5       0.34      0.20      0.25       256
           6       0.54      0.89      0.67       561
           7       0.58      0.07      0.13       149

    accuracy                           0.51      1099
   macro avg       0.26      0.17      0.16      1099
weighted avg       0.46      0.51      0.42      1099



c:\users\windows10\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\windows10\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\windows10\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [48]:
print(classification_report(c_Total_y_test, c_Total_y_pred))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         4
           3       0.12      0.03      0.05        31
           4       0.35      0.23      0.27        93
           5       0.40      0.26      0.31       236
           6       0.60      0.83      0.70       581
           7       0.27      0.14      0.19       152

    accuracy                           0.53      1099
   macro avg       0.25      0.21      0.22      1099
weighted avg       0.48      0.53      0.49      1099



c:\users\windows10\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\windows10\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\windows10\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
